In [26]:
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
import re

[nltk_data] Downloading package punkt to /Users/thanmai/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [27]:
from nltk.tokenize import sent_tokenize
sentences = []
i = 0
s = "Mahatma Gandhi was a great freedom fighter who spent his whole life in struggle for the independence of India. He was born in the Indian Hindu family on 2nd of October in 1869 in the Porbander, Gujarat. He lived his whole as a leader of the Indian people. His whole life story is a great inspiration for us. He is called as the Bapu or Rashtrapita as he spent his life in fighting against British rule for the freedom of us. While fighting with Britishers he took help of his great weapons like non-violence and Satyagraha movements to achieve freedom. Many times he got arrested and sent to the jail but he never discourages himself and continued fighting for national freedom.He is the real father of our nation who really used his all power to make us free from the British rule. He truly understood the power of unity in people (from different castes, religions, community, race, age or gender) which he used all through his independence movement. Finally he forced Britishers to quit India forever through his mass movements on 15th of August in 1947. Since 1947, the 15th of August is celebrated every year as the Independence Day in India.He could not continue his life after the independence of India in 1947 as he was assassinated by one of the Hindu activists, Nathuram Godse in 1948 on 30th of January. He was the great personality who served his whole life till death for the motherland. He enlightened our life with the true light of freedom from British rule. He proved that everything is possible with the non-violence and unity of people. Even after getting died many years ago, he is still alive in the heart of every Indian as a “Father of the Nation and Bapu”."
# for s in df['article_text']:
#    if i == 1:
#         break
s = re.sub(r"[^a-zA-Z0-9.]+", ' ', s)
s = s.lower()
sentences.append(sent_tokenize(s))
sentences = [y for x in sentences for y in x] # flatten list


In [28]:
sentences[:5]

['mahatma gandhi was a great freedom fighter who spent his whole life in struggle for the independence of india.',
 'he was born in the indian hindu family on 2nd of october in 1869 in the porbander gujarat.',
 'he lived his whole as a leader of the indian people.',
 'his whole life story is a great inspiration for us.',
 'he is called as the bapu or rashtrapita as he spent his life in fighting against british rule for the freedom of us.']

In [29]:
# Extract word vectors
word_embeddings = {}
def get_word_embeddings():
    f = open('static/glove.6B.100d.txt', encoding='utf-8')
    for line in f:
        values = line.split()
        word = values[0]

        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs
    f.close()
    return word_embeddings


In [30]:
# remove punctuations, numbers and special characters
# clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
 
# make alphabets lowercase
# clean_sentences = [s.lower() for s in clean_sentences]
clean_sentences = s

In [31]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thanmai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [32]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [33]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [34]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [35]:
# Extract word vectors
word_embeddings = {}
f = open('static/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [37]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)


In [38]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [41]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [42]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [43]:
# Extract top 10 sentences as the summary
for i in range(10):
  print(ranked_sentences[i][1])

he lived his whole as a leader of the indian people.
even after getting died many years ago he is still alive in the heart of every indian as a father of the nation and bapu .
finally he forced britishers to quit india forever through his mass movements on 15th of august in 1947. since 1947 the 15th of august is celebrated every year as the independence day in india.he could not continue his life after the independence of india in 1947 as he was assassinated by one of the hindu activists nathuram godse in 1948 on 30th of january.
he enlightened our life with the true light of freedom from british rule.
while fighting with britishers he took help of his great weapons like non violence and satyagraha movements to achieve freedom.
many times he got arrested and sent to the jail but he never discourages himself and continued fighting for national freedom.he is the real father of our nation who really used his all power to make us free from the british rule.
mahatma gandhi was a great freed